In [2]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from model import *
from train import *
from data import *
from constants import *
import math


def inference(model, dataloader: DataLoader, outfile, device):
    model.eval()
    all_probabilities = None
    for imgs, _ in tqdm(dataloader):
        imgs = imgs.to(device)
        predictions = model(imgs)  # each: [bsz, num_cls]
        if all_probabilities is None:
            all_probabilities = predictions
        else:
            all_probabilities = torch.cat((all_probabilities, predictions), dim=0)
    return all_probabilities


transform = inference_transform
BATCH_SIZE = 256

dl_tt = get_dataloader(
    f"/mnt/slurm_home/pzzhao/acad_projects/AI6102_proj/test_dataset",
    BATCH_SIZE,
    shuffle=False,
    transform=default_transform,
)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model_path = "/mnt/slurm_home/pzzhao/acad_projects/AI6102_proj/AI6102Project_ImageClassification/model_ckpt/resnet50_baseline/resnet50_2024-03-31_23-15_valLoss:0.862225.pth"
model_weights = torch.load(model_path, map_location=torch.device("cpu"))
# model = ResNetClassifier(num_classes=NUM_CLASSES)
# model.load_state_dict(model_weights)
model = model_weights
for param in model.parameters():
    param.requires_grad = False
model = model.to(device)
model.eval()
ret_pred = inference(model, dl_tt, None, device)
pass


cuda:0


100%|██████████| 510/510 [08:24<00:00,  1.01it/s]


In [3]:
import pandas as pd
import glob
import os
import torch.nn.functional as F

In [4]:

parent_folder = "/mnt/slurm_home/pzzhao/acad_projects/AI6102_proj/dataset/train"
subfolder_pattern = os.path.join(parent_folder, "*")
subfolders = glob.glob(subfolder_pattern)
subfolder_names = [os.path.basename(subfolder) for subfolder in subfolders]
subfolder_names.sort()

In [5]:
test_folder = f"/mnt/slurm_home/pzzhao/acad_projects/AI6102_proj/test_dataset/test/"
image_names = sorted(os.listdir(test_folder))

In [6]:
# predictions_np = ret_pred.detach().cpu().numpy()
predictions_np = F.softmax(ret_pred, dim=1).detach().cpu().numpy()
df = pd.DataFrame(predictions_np)
df.columns = subfolder_names
df.insert(0, 'image', image_names)

In [7]:
df.head(30)

,image,acantharia_protist,acantharia_protist_big_center,acantharia_protist_halo,amphipods,appendicularian_fritillaridae,appendicularian_s_shape,appendicularian_slight_curve,appendicularian_straight,artifacts,...,trichodesmium_tuft,trochophore_larvae,tunicate_doliolid,tunicate_doliolid_nurse,tunicate_partial,tunicate_salp,tunicate_salp_chains,unknown_blobs_and_smudges,unknown_sticks,unknown_unclassified
0,1.jpg,1.085782e-04,1.484740e-10,6.747295e-08,7.872458e-07,3.285130e-07,6.255255e-04,1.841697e-04,3.268667e-03,1.383242e-04,...,4.797008e-04,2.663192e-10,2.922593e-05,2.074897e-05,3.594987e-06,2.520306e-06,3.914967e-07,6.323696e-03,1.959405e-01,0.000260
1,10.jpg,1.338786e-02,2.020615e-06,2.559622e-04,3.800535e-03,6.845106e-07,5.415305e-04,2.278321e-03,5.113441e-03,1.352190e-04,...,6.114368e-04,1.037530e-07,4.417860e-05,2.212344e-05,1.832397e-05,7.267413e-06,5.360462e-07,6.815600e-02,1.703049e-03,0.024166
2,100.jpg,1.830404e-06,1.197761e-10,1.573390e-08,1.809346e-11,4.047659e-12,1.794386e-10,2.649172e-09,7.630408e-12,1.573280e-11,...,3.855170e-09,4.241061e-12,6.405000e-09,9.650774e-10,1.239446e-10,1.425681e-08,4.349130e-11,2.186730e-07,5.292073e-10,0.000009
3,1000.jpg,1.495715e-05,6.114016e-08,9.892386e-09,2.797813e-04,5.685018e-07,8.875445e-06,2.222637e-06,1.442174e-06,3.825375e-08,...,1.836312e-06,1.520648e-08,7.430475e-07,3.241089e-07,1.323220e-08,1.671490e-06,4.570060e-08,6.727221e-04,1.464648e-05,0.000329
4,10000.jpg,1.023699e-05,1.129210e-07,2.524817e-08,4.417230e-07,1.361943e-09,6.291485e-07,2.620985e-06,3.783660e-06,9.260813e-09,...,3.452062e-05,7.068257e-09,1.222074e-06,8.445111e-07,8.513336e-09,1.355627e-09,1.856336e-09,1.570710e-04,1.782556e-06,0.000149
5,100000.jpg,1.148601e-03,1.451629e-03,5.819164e-04,9.414937e-06,8.677538e-07,4.072369e-04,4.083955e-04,2.293735e-04,1.089523e-05,...,2.987174e-05,2.135348e-03,9.837061e-03,9.066541e-03,4.044913e-02,1.188001e-04,2.113243e-04,3.751887e-03,6.217179e-04,0.022920
6,100001.jpg,1.532780e-02,1.376316e-04,3.537446e-03,9.953244e-05,1.960450e-07,2.929238e-05,1.930409e-04,3.405815e-05,7.100564e-08,...,1.410680e-04,6.974448e-05,8.679834e-03,6.052343e-03,1.102778e-04,2.301676e-02,5.864248e-05,1.823249e-04,4.112457e-05,0.082583
7,100002.jpg,1.199299e-03,4.226799e-05,7.096539e-06,7.768171e-04,4.992001e-08,2.940171e-04,1.527438e-04,3.111475e-04,1.792457e-05,...,1.105587e-03,2.151906e-06,1.193378e-03,4.507849e-03,2.718530e-03,7.890257e-05,3.159589e-05,4.001616e-02,2.154914e-02,0.037838
8,100003.jpg,1.793212e-04,3.257158e-04,4.525102e-04,1.473753e-08,4.831446e-12,3.097357e-07,8.672565e-07,3.350078e-06,2.318723e-07,...,2.259920e-04,6.636952e-06,1.480621e-03,3.241975e-04,5.616329e-04,1.152997e-05,5.860618e-05,1.288862e-04,1.594330e-05,0.009856
9,100004.jpg,4.280015e-02,2.065205e-06,1.468466e-06,4.814926e-07,4.666223e-10,1.950998e-06,4.721093e-06,1.142708e-05,8.476393e-04,...,5.345318e-06,9.119943e-08,4.622084e-06,1.649759e-05,4.387001e-07,1.415240e-07,4.135537e-07,5.882248e-05,2.122485e-04,0.000280


In [8]:
df.to_csv('resnet50_baseline.csv', index=False)